# Tutorial: NGEM - Neural graph error mitigation

In [3]:
import os
from pathlib import Path
from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit.providers.fake_provider import FakeLimaV2
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator, Estimator

from tqdm.notebook import tqdm_notebook

from blackwater.data.loaders.dataclasses import ExpValDataSet as ExpValData
from blackwater.data.dataio import ExpValDataWriter

ImportError: cannot import name 'ExpValData' from 'blackwater.data.loaders.dataclasses' (/Users/mark/Documents/Quantum/miti-learn/.venv/lib/python3.9/site-packages/blackwater/data/loaders/dataclasses.py)

In [ ]:
fake_lima = FakeLimaV2()

ideal_estimator = Estimator()
noisy_esimator = BackendEstimator(fake_lima)

data_folder = "./data/ngem/"
Path(data_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
# generate data

N_FILES = 10
N_ENTRIES_PER_FILE = 100
N_QUBITS = 3
DEPTH = 3

operator = SparsePauliOp(["ZZZZZ", "ZIIIZ"])
writer = ExpValDataWriter()

for idx in tqdm_notebook(range(N_FILES)):
    
    entries = []
    for _ in range(N_ENTRIES_PER_FILE):
        
        circuit = random_circuit(N_QUBITS, DEPTH)
        transpiled_circuit = transpile(circuit, fake_lima)
        
        ideal_exp_val = ideal_estimator.run([transpiled_circuit], [operator]).result().values[0]
        noisy_exp_val = noisy_esimator.run([transpiled_circuit], [operator]).result().values[0]

        exp_val_data = ExpValData.build(
            circuit=transpiled_circuit,
            expectation_values=[
                ideal_exp_val,
                noisy_exp_val
            ],
            observable=operator,
            backend=fake_lima
        )
        entries.append(exp_val_data)
    
    save_path = os.path.join(data_folder, f"{idx}_entries.json")
    writer.save_to_file(save_path, entries)

In [ ]:
# load data

import glob
from torch_geometric.loader import DataLoader
from blackwater.data.loaders.dataclasses import ExpValDataSet

paths = glob.glob(f"{data_folder}/**.json")
split = int(0.6 * len(paths))
train_paths, val_paths = paths[:split], paths[split:]

BATCH_SIZE = 32

train_loader = DataLoader(ExpValDataSet(train_paths), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ExpValDataSet(val_paths), batch_size=BATCH_SIZE, shuffle=False)

for data in train_loader:
    print(data)
    break

In [ ]:
# create model

from blackwater.library.ngem import SimpleExpectationValueModel
"""
model = SimpleExpectationValueModel(
    num_node_features=22, 
    num_obs_features=21,
    hidden_channels=10
)
"""

In [ ]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from miti_learn.models import qiskit_blackwater as miti_models

criterion = torch.nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 
                              'min', 
                              factor=0.1, 
                              patience=15, 
                              verbose=True, 
                              min_lr=0.00001)

mitigator = miti.MitiLearn()
model = miti_models.SimpleExpectationValueModel(
    num_node_features=22, 
    num_obs_features=21,
    hidden_channels=10
)


In [ ]:
# train
import numpy as np

min_valid_loss = np.inf


train_losses = []
val_losses = []

N_EPOCHS = 10

progress = tqdm_notebook(range(N_EPOCHS), desc='Model training', leave=True)
for epoch in progress:
    train_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        ideal_exp_val, noisy_exp_val = torch.tensor_split(data.y, 2, dim=1)
        out = model(data.x, data.edge_index, noisy_exp_val, data.observable, data.circuit_depth, data.batch)
        loss = criterion(out, ideal_exp_val)     
        
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    valid_loss = 0.0
    model.eval()
    for i, data in enumerate(val_loader):
        ideal_exp_val, noisy_exp_val = torch.tensor_split(data.y, 2, dim=1)
        out = model(data.x, data.edge_index, noisy_exp_val, data.observable, data.circuit_depth, data.batch)
        loss = criterion(out, ideal_exp_val)        

        valid_loss += loss.item()

    scheduler.step(valid_loss)

    if epoch >= 1:    
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(valid_loss / len(val_loader))

        progress.set_description(f"{round(train_losses[-1], 5)}, {round(val_losses[-1], 5)}")
        progress.refresh()

In [ ]:
# visualize

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.plot(train_losses, label="train_loss")
plt.plot(val_losses, label="val_loss")
plt.yscale('log')

plt.legend()
plt.show()

distances = []
for i, data in enumerate(val_loader):
    ideal_exp_val, noisy_exp_val = torch.tensor_split(data.y, 2, dim=1)
    out = model(data.x, data.edge_index, noisy_exp_val, data.observable, data.circuit_depth, data.batch)
        
    for ideal, noisy, ngm_mitigated in zip(
        ideal_exp_val.tolist(),
        noisy_exp_val.tolist(),
        out.tolist()
    ):
        ideal = np.mean(ideal)
        noisy = np.mean(noisy)
        ngm_mitigated = np.mean(ngm_mitigated)
        distances.append({
            "ideal": ideal,
            "noisy": noisy,
            "ngm_mitigated": ngm_mitigated,
            "dist_noisy": np.abs(ideal - noisy),
            "dist_ngm": np.abs(ideal - ngm_mitigated),
        })
        
df = pd.DataFrame(distances)
sns.boxplot(data=df[["dist_noisy", "dist_ngm"]], orient="h", showfliers = False)
plt.title("Dist to ideal exp value")
plt.show()

sns.histplot([df['ideal'], df['noisy'], df["ngm_mitigated"]], kde=True, bins=40)
plt.title("Exp values distribution")
plt.show()

In [ ]:
# estimator

from blackwater.primitives.learning_esimator import (
    learning_estimator,
    ScikitLearnEstimatorModel,
    TorchGeometricEstimatorModel,
)
from blackwater.data.encoders.torch import DefaultPyGEstimatorEncoder


circuit = transpile(random_circuit(5, 2),  fake_lima)
observable = SparsePauliOp(["ZZZZZ", "ZIIIZ"])

estimator = learning_estimator(
    Estimator, 
    model=TorchGeometricEstimatorModel(model, DefaultPyGEstimatorEncoder()), 
    backend=fake_lima, 
    skip_transpile=True
)()

In [ ]:
estimator.run([circuit], [observable]).result()